<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/DDI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [16]:
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras

## CONFRONTO TRAINING SET E TEST SET

In [4]:
dev = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/train.tsv', sep='\t')

In [12]:
print(dev.shape)
print(test.shape)
print(train.shape)

(7244, 3)
(5761, 3)
(18779, 3)


Si osserva che le proporzioni sono costanti circa

In [14]:
print(dev.label.value_counts())
print(test.label.value_counts())
print(train.label.value_counts())

DDI-false        6240
DDI-effect        396
DDI-mechanism     373
DDI-advise        193
DDI-int            42
Name: label, dtype: int64
DDI-false        4782
DDI-effect        360
DDI-mechanism     302
DDI-advise        221
DDI-int            96
Name: label, dtype: int64
DDI-false        15842
DDI-effect        1212
DDI-mechanism      946
DDI-advise         633
DDI-int            146
Name: label, dtype: int64


## OSSERVAZIONI SUL DATASET

In [41]:
df.head(5)

,index,sentence,label
0,DDI-DrugBank.d0.s0.p0,The concomitant intake of @DRUG$ and @DRUG$ does not affect the pharmacokinetics of either alcohol or acamprosate.,DDI-false
1,DDI-DrugBank.d0.s0.p1,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or acamprosate.,DDI-false
2,DDI-DrugBank.d0.s0.p2,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either alcohol or @DRUG$.,DDI-false
3,DDI-DrugBank.d0.s0.p3,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either @DRUG$ or acamprosate.,DDI-false
4,DDI-DrugBank.d0.s0.p4,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either alcohol or @DRUG$.,DDI-false


In [5]:
df = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/train.tsv', sep='\t')
print(f"COLONNE:{df.columns}")
print(f"DIMENSIONE:{df.shape}")

COLONNE:Index(['index', 'sentence', 'label'], dtype='object')
DIMENSIONE:(18779, 3)


In [37]:
df.label.unique()

array(['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise',
       'DDI-int'], dtype=object)

In [40]:
df.label.value_counts()

DDI-false        15842
DDI-effect        1212
DDI-mechanism      946
DDI-advise         633
DDI-int            146
Name: label, dtype: int64

In [38]:
df.loc[range(10), ['sentence']]

,sentence
0,The concomitant intake of @DRUG$ and @DRUG$ does not affect the pharmacokinetics of either alcohol or acamprosate.
1,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or acamprosate.
2,The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either alcohol or @DRUG$.
3,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either @DRUG$ or acamprosate.
4,The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either alcohol or @DRUG$.
5,The concomitant intake of alcohol and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or @DRUG$.
6,Co-administration of @DRUG$ with @DRUG$ produced a 25% increase in AUC and a 33% increase in the Cmax of acamprosate.
7,Co-administration of @DRUG$ with Acamprosate produced a 25% increase in AUC and a 33% increase in the Cmax of @DRUG$.
8,Co-administration of naltrexone with @DRUG$ produced a 25% increase in AUC and a 33% increase in the Cmax of @DRUG$.
9,The pharmacokinetics of @DRUG$ and its major metabolite @DRUG$ were unaffected following co-administration with Acamprosate.


## FASE PRELIMINARE MODELLO

In [17]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/index2word.pkl", 'rb') as output:
  i2w = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

In [51]:
five_hot = OneHotEncoder(sparse=False).fit_transform(
  df.label.to_numpy().reshape(-1, 1)
)

In [60]:
def my_text_to_word_sequence(sentence):
  return keras.preprocessing.text.text_to_word_sequence(sentence,
                                                        filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                        lower=True)

sentences = [my_text_to_word_sequence(sentence) for sentence in df['sentence']]

In [65]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)

print(f'Il massimo è {max} in corrispondenza di {max_index}')

Il massimo è 82 in corrispondenza di 17771


In [ ]:
embedded_trainset = np.zeros(shape=(len(sentences), max, 300))
for i, sentence in enumerate(sentences):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

## CREAZIONE MODELLO

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.LSTM(64, recurrent_dropout=0.2))
model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

In [ ]:
result = model.fit(embedded_trainset, five_hot, epochs=2, batch_size=25)